<a href="https://colab.research.google.com/github/TOM-BOHN/SFDC-User-Permissions-AI/blob/main/Notebooks/SFDC_User_Permission_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

Install the Python SDK.

In [1]:
!git clone https://github.com/TOM-BOHN/SFDC-User-Permissions-AI.git

Cloning into 'SFDC-User-Permissions-AI'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 154 (delta 69), reused 98 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (154/154), 58.88 KiB | 1.23 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [2]:
!pip install -Uq "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.9 MB/s eta 0:00:00


In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

genai.__version__

###################################

import sys
import os
import time

import enum
import json

import pandas as pd

###################################

os.chdir('/content/SFDC-User-Permissions-AI')

###################################

# Import the processing functions
from src.processing import extract_json_fields
from src.utils.data_utils import save_data
from src.llms import RiskRating, eval_summary, create_chat_session, classify_risk_rating


### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [4]:
from google.colab import userdata

client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

### Automated retry

This codelab sends a lot of requests, so set up an automatic retry
that ensures your requests are retried when per-minute quota is reached.

In [5]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [6]:
url = "https://raw.githubusercontent.com/TOM-BOHN/SFDC-User-Permissions-AI/refs/heads/main/data/input/User_Permission_Reference_Data__Sample.csv"
perm_list_df = pd.read_csv(url)
perm_list_df.head()

,Permission Name,API Name,Description
0,Access Data Cloud Data Explorer,AccessCdpDataExplorer,Allows user access Data Cloud Data Explorer.
1,Administer territory operations,ManageTerritories,Prerequisite user permission for a user to man...
2,Allow sending of List Emails,ListEmailSend,"Allow users to create, edit and send List Emails"
3,Api Only User,ApiUserOnly,Access Salesforce.com only through a Salesforc...
4,Author Apex,AuthorApex,Create Apex classes and triggers.


In [7]:
with open('/content/SFDC-User-Permissions-AI/src/prompts/templates/prompt_user_perm_risk_rating.md', 'r') as f:
    PROMPT_USER_PERM_RISK_RATING = f.read()

print(PROMPT_USER_PERM_RISK_RATING)

# Permission Risk Evaluation Prompt Template  
# --------------------------------------------------
# This template can be imported and formatted with the specific
# `permission_name` and `permission_description` variables to create
# a concrete evaluation prompt for any Salesforce permission.
# --------------------------------------------------

PERMISSION_RISK_PROMPT = """
# Instruction
You are a **Salesforce security risk assessor**.
Your task is to evaluate the **inherent risk level** of a Salesforce permission (or capability) when granted to a user.
We will provide you with the permission name and a short description of what it allows.
Analyze the permission against the **Evaluation Criteria** below and assign one of the five **Risk Levels** defined in the Rating Rubric.
Give step‑by‑step reasoning for your decision, citing the specific criteria that most influenced your rating.

# Evaluation

## Metric Definition
**Permission Risk** [aka weighted_score] measures the potential neg

In [8]:
chat_session = create_chat_session(client = client, model_name='gemini-2.0-flash')

# Evaluate a single permission
text_eval, struct_eval = eval_summary(
    prompt=PROMPT_USER_PERM_RISK_RATING,
    name=perm_list_df['Permission Name'][0],
    api_name=perm_list_df['API Name'][0],
    description=perm_list_df['Description'][0],
    model_name='gemini-2.0-flash',
    client=client,
    chat_session=chat_session  # Reuse the same session
)

# Display the result
Markdown(text_eval)
print(f"Risk Rating: {struct_eval.name} ({struct_eval.value})")

Risk Rating: SENSITIVE (3)


In [9]:
results_df = classify_risk_rating(
      input_df = perm_list_df
    , prompt = PROMPT_USER_PERM_RISK_RATING
    , chat_session = chat_session
    , total_records = 2
    , checkin_interval = 60
    , debug = True
  )
results_df

Starting job to process 2 records.
####################

Analyzing Permission 1 of 2...
Name:        Access Data Cloud Data Explorer
API Name:    AccessCdpDataExplorer
Description: Allows user access Data Cloud Data Explorer.
--------------------
Risk Rating: RiskRating.SENSITIVE
####################

Analyzing Permission 2 of 2...
Name:        Administer territory operations
API Name:    ManageTerritories
Description: Prerequisite user permission for a user to manage a territory branch.
--------------------
Risk Rating: RiskRating.SENSITIVE
####################


####################
Total time taken: 4.20 seconds to process 2 records.
Average time per record: 2.10 seconds

Sample Output of Results:
                   Permission Name               API Name  \
0  Access Data Cloud Data Explorer  AccessCdpDataExplorer   
1  Administer territory operations      ManageTerritories   

                                         Description           Risk Rating  \
0       Allows user access D

,Permission Name,API Name,Description,Risk Rating,Evaluation,Processing Time
0,Access Data Cloud Data Explorer,AccessCdpDataExplorer,Allows user access Data Cloud Data Explorer.,RiskRating.SENSITIVE,"```json\n{\n ""risk_tier"": ""Sensitive"",\n ""ri...",1.975064
1,Administer territory operations,ManageTerritories,Prerequisite user permission for a user to man...,RiskRating.SENSITIVE,"```json\n{\n ""risk_tier"": ""Sensitive"",\n ""ri...",2.214836


In [10]:
full_results_df = extract_json_fields(
    results_df
  , json_column='Evaluation'
  , debug = True
)


First 5 rows of processed data:


,Permission Name,API Name,Description,Risk Rating,Evaluation,Processing Time,Risk Tier,Weighted Score,Scores,Rationale,Confidence
0,Access Data Cloud Data Explorer,AccessCdpDataExplorer,Allows user access Data Cloud Data Explorer.,3,"{ ""risk_tier"": ""Sensitive"", ""risk_rating"": ""...",1.975064,Sensitive,3.1,"{'Data_Sensitivity': 3, 'Scope_of_Impact': 4, ...",Access to Data Cloud Data Explorer grants acce...,High
1,Administer territory operations,ManageTerritories,Prerequisite user permission for a user to man...,3,"{ ""risk_tier"": ""Sensitive"", ""risk_rating"": ""...",2.214836,Sensitive,2.7,"{'Data_Sensitivity': 3, 'Scope_of_Impact': 3, ...",Administering territory operations allows mana...,High



Columns added: ['Risk Tier', 'Risk Rating', 'Weighted Score', 'Scores', 'Rationale', 'Confidence']


In [11]:
results_df['Evaluation'][1]

'```json\n{\n  "risk_tier": "Sensitive",\n  "risk_rating": "3",\n  "weighted_score": 2.7,\n  "scores": {\n    "Data_Sensitivity": 3,\n    "Scope_of_Impact": 3,\n    "Configurational_Authority": 3,\n    "External_Data_Exposure": 2,\n    "Regulatory_Obligation": 3,\n    "Segregation_of_Duties": 3,\n    "Auditability": 2,\n    "Reversibility": 2\n  },\n  "rationale": "Administering territory operations allows management of a territory branch, implying control over assignment of accounts and opportunities, which can impact data visibility and sales forecasting. This can lead to potential regulatory concerns and segregation of duties issues depending on how territories are structured and how sensitive data is distributed. Poor auditability and difficult reversibility further elevate the risk.",\n  "confidence": "High"\n}\n```'

In [12]:
full_results_df.iloc[0].to_dict()

{'Permission Name': 'Access Data Cloud Data Explorer',
 'API Name': 'AccessCdpDataExplorer',
 'Description': 'Allows user access Data Cloud Data Explorer.',
 'Risk Rating': '3',
 'Evaluation': '{  "risk_tier": "Sensitive",  "risk_rating": "3",  "weighted_score": 3.1,  "scores": {    "Data_Sensitivity": 3,    "Scope_of_Impact": 4,    "Configurational_Authority": 2,    "External_Data_Exposure": 3,    "Regulatory_Obligation": 3,    "Segregation_of_Duties": 3,    "Auditability": 3,    "Reversibility": 2  },  "rationale": "Access to Data Cloud Data Explorer grants access to potentially sensitive data within Data Cloud, impacting a broad scope of data depending on the connected sources and configurations. While not inherently granting modification capabilities, exposure and potential export of sensitive information raise regulatory obligation concerns.  The lack of inherent configuration authority and limited direct impact on core system configurations mitigate the overall risk somewhat.",  

In [13]:
# Save the results DataFrame
save_data(
    data=full_results_df,
    filename='results',
    data_type='output',  # This will save to data/output/
    format='csv',
    index=False
)

'data/output/results.csv'